# ĐỒ ÁN THỰC HÀNH
# CSC17104 – LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU

Giảng viên: Bùi Tiến Lên

Thông tin thành viên:
1. Nguyễn Văn Tuấn Đạt 19120472
2. Phan Xuân Hoài 20120481
3. Luân Mã Khương 20120515
4. 

## Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
sns.set_theme()

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)

## A. Thu thập dữ liệu
- Tên dữ liệu: HR Analytics: Job Change of Data Scientists
- Nguồn lấy dữ liệu: kaggle
- Link dữ liệu: https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists
- License: CC0: Public Domain
- Mô tả về dữ liệu: dữ liệu gồm tập train và tập test. Trong đồ án này, nhóm sẽ sử dụng tập train.
            + Dữ liệu gồm 14 cột và 19158 dòng.

In [ ]:
df = pd.read_csv('aug_train.csv', encoding='utf-8')
df.shape

In [ ]:
df.head(10)

## B. Khám phá dữ liệu (thường đan xen với pha tiền xử lý dữ liệu)

### 1.Mỗi dòng có ý nghĩa gì? Có vấn đề với các dòng có ý nghĩa khác nhau không?

* Mỗi dòng là thông tin của từng ứng viên, trong đó mỗi ô của dòng là dữ liệu tương ứng với cột của ô đó
* Không có vấn đề các dòng có ý nghĩa khác nhau

### 1.1 Kiểm tra các dòng có bị trùng lặp hay không?

In [ ]:
have_duplicated_rows = np.any(df.duplicated())
have_duplicated_rows

Các dòng không bị trùng lặp.

### 2.Mỗi cột có ý nghĩa gì?
            
            - enrollee_id : ID với từng ứng viên

            - city: Mã thành phố

            - city_ development _index : Chỉ số phát triển của thành phố (theo tỷ lệ)

            - gender: Giới tính

            - relevent_experience: Kinh nghiệm liên quan của ứng viên

            - enrolled_university: Loại khóa học đại học đã đăng ký nếu có

            - education_level: Trình độ học vấn

            - major_discipline: Ngành học chính

            - experience: Kinh nghiệm (tính theo năm)

            - company_size: Số lượng nhân viên trong công ty của người sử dụng lao động hiện tại

            - company_type : Loại chủ lao động hiện tại

            - last_new_job: Sự khác biệt về số năm giữa công việc trước đây và công việc hiện tại

            - training_hours: Thời gian hoàn thành huấn luyện

            - target: 0 – Không nhảy việc, 1 – Nhảy việc

### 3.Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

In [ ]:
df.dtypes

In [ ]:
df['experience'].unique()

Vì trong feature experience có các dữ liệu như <1, 1, 2, ..., 20, >20 nên ta sẽ chuyển dữ liệu sang dạng các khoảng như <1, 1-5, 6-10, 11-15, 16-20, >20

In [ ]:
df['experience'].replace({'1':'1-5',
                            '2':'1-5',
                            '3':'1-5',
                            '4':'1-5',
                            '5':'1-5',
                            '6':'6-10',
                            '7':'6-10',
                            '8':'6-10',
                            '9':'6-10',
                            '10':'6-10',
                            '11':'11-15',
                            '12':'11-15',
                            '13':'11-15',
                            '14':'11-15',
                            '15':'11-15',
                            '16':'16-20',
                            '17':'16-20',
                            '18':'16-20',
                            '19':'16-20',
                            '20':'16-20'},inplace=True)

### 4.Với mỗi cột, các giá trị được phân bố như thế nào?

#### 4.1 Với các cột có giá trị dạng số

**Các cột dạng số:** _'enrollee_id', 'city_development_index', 'training_hours', 'target'_ 

DataFrame `summary_df` bên dưới chứa thông tin phân bố giá trị của các cột trên:
- Tên của các cột là tên của các cột số trong `df`
- Tên của các dòng có ý nghĩa là: 
 + `"missing_ratio"`: tỉ lệ phần trăm các giá trị thiếu
 + `"count"`: số lượng giá trị 
 + `"mean"`: giá trị trung bình
 + `"std"`: độ lệch chuẩn
 + `"min"`: giá trị nhỏ nhất
 + `"25%"`: giá trị phân vị 25%
 + `"50%"`: giá trị phân vị 50%
 + `"75%"`: giá trị phân vị 75%
 + `"max"`: giá trị lớn nhất

In [ ]:
summary_df = df.describe().round(1)
missing_ratio = pd.DataFrame(df[['enrollee_id', 'city_development_index', 'training_hours', 'target']].isna().sum() * 100/ len(df)).T.round(1)
missing_ratio = missing_ratio.set_axis(['missing_ratio'])
summary_df = pd.concat([missing_ratio, summary_df])
summary_df

Theo quan sát ở trên, ta thấy các cột có giá trị số không có giá trị thiếu. Mọi thứ trong có vẻ ổn.

### 4.2 Với các cột có giá trị categorical

Các giá trị sẽ tính:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu 
- Số lượng các giá trị (các giá trị ở đây là các giá trị khác nhau và ta không xét giá trị thiếu)
- Tỉ lệ % (từ 0 đến 100) của mỗi giá trị được sort theo tỉ lệ % giảm dần (ta không xét giá trị thiếu, tỉ lệ là tỉ lệ so với số lượng các giá trị không thiếu)

In [ ]:
def missing_ratio(x):
    return (x.isna().mean()*100)
def num_values(x):
    return len(x.dropna().value_counts())
def value_ratios(x):
    count_value =  x.dropna().value_counts().sort_values(ascending=False)
    return (count_value/count_value.sum()*100).round(1).to_dict()
cat_df_info = df.select_dtypes(include="object").agg(func=[missing_ratio, num_values, value_ratios])
cat_df_info

Các cột gender, company_size, company_type đều có số lượng giá trị thiếu khá lớn, trên 22%. Các số liệu khác trong có vẻ ổn

### 5. Tiền xử lí dữ liệu

In [ ]:
df.info()

Số lượng giá trị bị thiếu.

In [ ]:
df.isnull().sum()

Ta thấy có khá nhiều giá trị bị thiếu và đa số đều là categorical. Vì vậy, ta sẽ xóa các dòng bị thiếu.

In [ ]:
#vì các dòng "relevant_experience" = "No relevent experience" thì các giá trị "company_size" và "company_type" là NaN. 
# nên ta cần giữ những dòng này lại 
df[df["relevent_experience"]=="No relevent experience"] = df[df["relevent_experience"]=="No relevent experience"].replace(np.nan, "None")
df.dropna(inplace=True)
df

Số lượng dòng bị thiếu khá nhiều, khoảng 1/2 của dữ liệu.

## C. Đưa ra các câu hỏi có ý nghĩa cần trả lời

### 1. Số lượng Data Scientists trên 10 năm kinh nghiệm đang tìm kiếm công việc mới trong Top 5 thành phố phát triển nhất?

Lợi ích khi trả lời được câu hỏi:
- Với vai trò là doanh nghiệp sẽ biết thêm thông tin để thay đổi chính sách đãi ngộ hay điều chỉnh môi trường làm việc để giữ chân các nhân viên tài năng của minh.
- Liệu các Data Scientists nhiều năm kinh nghiệm có nhu cầu tìm kiếm thách thức mới cho bản thân mình.
- Thị trường việc làm về Data Scientist ở trong Top 5 thành phố phát triển nhất.

### 2. Số năm kinh nghiệm và kinh nghiệm có liên quan đến DS ảnh hưởng thế nào đến quyết định có nhảy việc không của người tham gia đào tạo?

**Lợi ích khi trả lời được câu hỏi:** Khi trả lời được câu hỏi công ty có thể đánh giá được đâu là một ứng viên tiềm năng muốn làm việc cho công ty dựa vào số năm kinh nghiệm và kinh nghiệm có liên quan đến DS của ứng viên để 

## D. Tiền xử lý và phân tích dữ liệu để trả lời cho từng câu hỏi

### 1. Số lượng Data Scientists trên 10 năm kinh nghiệm đang tìm kiếm công việc mới trong Top 5 thành phố phát triển nhất?

### 2. Số năm kinh nghiệm và kinh nghiệm có liên quan đến DS ảnh hưởng thế nào đến quyết định có nhảy việc không của người tham gia đào tạo?

### 3. Có sự chênh lệch trình độ học vấn giữa nam và nữ hay không? Điều này có ảnh hưởng tới chỉ số phát triển của thành phố nơi mà họ làm việc?

<h3 style='color:red; font-size:25px'> 2.1 Tương quan giữa số năm kinh nghiệm và quyết định nhảy việc </h3>

<h3 style='color:#6ac61f'> Bước 1: Tính phần trăm người quyết định nhảy việc so với tổng số người trong cùng khoảng số năm kinh nghiệm </h3>

+ 1. Tính số lượng người trong mỗi khoảng năm kinh nghiệm vào series `experience_total`
+ 2. Tính số lượng người có quyết định nhảy việc vào series `experience_change`
+ 3. Tính phần trăm số lượng người quyết định nhảy việc so với tổng số người trong cùng một khoảng số năm kinh nghiệm vào series `experience_change_percent`

In [ ]:
#Tính số lượng người trong mỗi khoảng năm kinh nghiệm
experience_total = df.groupby('experience')['experience'].size()

#Tính số lượng người có quyết định nhảy việc
mask = df['target'] == 1 
experience_change = df[mask].groupby('experience')['experience'].size()

#Tính phần trăm 
experience_change_percent = experience_change/experience_total

experience_change_percent

<h3 style='color:#6ac61f'> Bước 2: Trực quan </h3>

Ta sẽ trực quan mối tương quan giữa số năm kinh nghiệm và quyết định nhảy việc bằng đồ thị dạng cột, với:
+ Trục hoành thể hiện số năm kinh nghiệm
+ Trục tung thể hiện phần trăm số người quyết định nhảy việc so với tổng số người trong cùng một khoảng số năm kinh nghiệm

In [ ]:
plt.figure(figsize=(10,7))

sns.barplot(x = experience_change_percent.index, y = experience_change_percent.values)

plt.title('Phần trăm số người nhảy việc so với tổng số người trong cùng khoảng số năm kinh nghiệm', fontsize=14)
plt.xlabel('Số năm kinh nghiệm', fontsize=14)
plt.ylabel('Phần trăm(%)', fontsize=14)
plt.grid(True)

plt.show()

<h3 style='color:#6ac61f; line-height: 1.5;'>🤔🤔🤔 Từ biểu đồ trên ta có thể thấy một ứng viên có số năm kinh nghiệm càng cao thì khả năng ứng viên đó nhảy việc càng thấp. Điều này có thể dự đoán là bởi khi một người có số năm kinh nghiệm và tuổi càng tăng lên họ sẽ muốn một công việc ổn định bởi cơ hội và khả năng thăng tiến cũng giảm dần.
</h3>

---

<h3 style='color:red; font-size:25px'> 2.2 Tương quan giữa kinh nghiệm liên quan đến DS và quyết định nhảy việc </h3>

<h3 style='color:#6ac61f'> Bước 1: Tính phần trăm nhảy việc so với tổng số người có kinh nghiệm hoặc không có kinh nghiệm liên quan đến DS </h3>

+ 1. Tính số lượng người có kinh nghiệm và không có kinh nghiệm DS vào series `sr_total`
+ 2. Tính số lượng người có quyết định nhảy việc có kinh nghiệm và không có kinh nghiệm DS vào series `sr_change`
+ 3. Tính phần trăm nhảy việc so với tổng số người có kinh nghiệm hoặc không có kinh nghiệm liên quan đến DS vào series `sr_change_percent`

In [ ]:
#Tính số lượng người có kinh nghiệm và không có kinh nghiệm DS 
sr_total = df[['relevent_experience']].groupby('relevent_experience').size()

#Tính số lượng người có quyết định nhảy việc có kinh nghiệm và không có kinh nghiệm DS
mask = df['target'] == 1
sr_change = df.loc[mask,['relevent_experience']].groupby('relevent_experience').size()

#Tính phần trăm nhảy việc so với tổng số người có kinh nghiệm hoặc không có kinh nghiệm liên quan đến DS
sr_change_percent = sr_change/sr_total * 100
sr_change_percent

<h3 style='color:#6ac61f'> Bước 2: Trực quan </h3>

Ta sẽ trực quan mối tương quan giữa kinh nghiệm liên quan đến DS và quyết định nhảy việc bằng đồ thị dạng cột, với:
+ Trục hoành gồm 2 cột `Has relevent experience` và `No relevent experience`
+ Trục tung thể hiện phần trăm

In [ ]:
plt.figure(figsize=(10,7))

sns.barplot(x=sr_change_percent.index, y=sr_change_percent.values, color="#6ac61f")

plt.title('Tương quan giữa kinh nghiệm liên quan đến DS và quyết định nhảy việc', fontsize=12)
plt.xlabel('Kinh nghiệm liên quan đến DS', fontsize=12)
plt.ylabel('Phần trăm(%)', fontsize=12)
plt.grid(True)

plt.show()

<h3 style='color:#6ac61f; line-height: 1.5;'>🧐🧐🧐 Từ biểu đồ trên ta có thể thấy phần trăm những người không có kinh nghiệm liên quan đến DS có mong muốn nhảy việc cao hơn những người có kinh nghiệm khoảng 10%.
</h3>

<h3 style='color:red; font-size:25px'> 2.3 Sự chênh lệch giữa trình độ học vấn và mức sống phân nhóm theo giới tính </h3>

Thống kê số lượng các ứng viên theo giới tính

In [ ]:
num_applicants_by_gender = df.groupby('gender').size()
num_applicants_by_gender

Ta thấy số lượng ứng viên nam có 1 sự chênh lệch rõ rệt so với số lượng ứng viên với các giới tính khác<br>
Để tìm hiểu sâu hơn ta tiến hành thống kê trình độ học vấn của các ứng viên theo giới tính

In [ ]:
studies_by_gender = df.groupby(['gender', 'education_level']).size()
studies_by_gender

DataFrame ```education_ratio``` gồm 4 cột ứng với giới tính và 6 dòng ứng với trình độ học vấn lưu tỉ lệ trình độ học vấn theo giới tính (từ 0 đến 100)

In [ ]:
education_ratio = studies_by_gender / num_applicants_by_gender * 100
education_ratio = education_ratio.to_frame()
education_ratio = education_ratio.reset_index().pivot(index = 'education_level', columns = 'gender')
education_ratio = education_ratio.where(education_ratio.notna(), 0)
education_ratio = education_ratio[0]
education_ratio

Trực quan hóa bảng dữ liệu trên thành biểu đồ cột

In [ ]:
education_ratio.plot.bar()

Như vậy ta thấy nữ giới có tỉ lệ trình độ học vấn trên THPT là cao nhất mặc dù có 1 sự chênh lệch lớn giữa số lượng so với nam giới<br>
Liệu các ứng viên nữ có làm việc ở các thành phố có mức sống cao hơn hay không?

Ta xem xét trung bình mức sống của các thành phố mà các ứng viên làm việc

In [ ]:
city_development_index_mean = df.groupby(['gender']).mean()
city_development_index_mean['city_development_index']

Mặc dù nữ giới có tỉ lệ trình độ học vấn cao nhất nhưng các ứng viên thuộc giới tính khác lại là người có mức sống cao nhất
mặc dù không có ứng viên nào có trình độ Phd

--- ----

## E. Tổng hợp lại quá trình thực hiện đồ án